<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week8_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 8 Practical (Python, Colab)
**Topic:** Logic & declarative programming — unification and SLD resolution (mini‑Prolog)
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO - ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This PH builds a tiny Prolog‑like engine in Python: terms, unification with occurs‑check, and SLD resolution over Horn clauses. You will run queries over a small family knowledge base and write one new relation.

### Learning goals (about 60 minutes)
- Represent first‑order **terms** (variables, atoms, functors) and **substitutions**.
- Implement **unification** (with occurs‑check) and **SLD resolution** for Horn clauses.
- Run queries over a small knowledge base (KB) and extend it with a new relation.

In [ ]:
# Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# Environment check (Python 3.10+ for match/case)
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — match/case available.")

In [ ]:
# ===============================
# Part 1 — Terms and substitutions
# ===============================
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple, Iterable, Iterator, Optional, Set, Union

@dataclass(frozen=True) class Var:   name: str
@dataclass(frozen=True) class Atom:  name: str
@dataclass(frozen=True) class Fun:   name: str; args: Tuple["Term", ...]
Term = Var | Atom | Fun

Subst = Dict[str, Term]  # maps variable name -> Term

def is_var(t: Term) -> bool: return isinstance(t, Var)

def deref(t: Term, s: Subst) -> Term:
    # Follow substitution links until t is not a bound Var
    while isinstance(t, Var) and t.name in s:
        t = s[t.name]
    return t

def ftv(t: Term, s: Subst | None = None) -> Set[str]:
    # Free type variables in a term (here: just variable names), after deref
    t = deref(t, s or {})
    if isinstance(t, Var):  return {t.name}
    if isinstance(t, Atom): return set()
    if isinstance(t, Fun):
        out: Set[str] = set()
        for a in t.args:
            out |= ftv(a, s or {})
        return out
    raise TypeError("unknown term")

In [ ]:
# ===============================
# Part 2 — Occurs check and unification
# ===============================
class UnifyError(Exception): pass

def occurs(x: str, t: Term, s: Subst) -> bool:
    # Occurs check: does variable x occur in t (after deref)?
    t = deref(t, s)
    if isinstance(t, Var):  return t.name == x
    if isinstance(t, Atom): return False
    if isinstance(t, Fun):
        return any(occurs(x, a, s) for a in t.args)
    raise TypeError("unknown term")

def bind(x: str, t: Term, s: Subst) -> Subst:
    # Bind variable x to term t with occurs check
    t = deref(t, s)
    if isinstance(t, Var) and t.name == x:  # x = x
        return s
    if occurs(x, t, s):
        raise UnifyError(f"occurs check failed: {x} in {t}")
    s2 = dict(s); s2[x] = t
    return s2

def unify(t1: Term, t2: Term, s: Subst | None = None) -> Subst:
    # Robinson unification with occurs check
    s = {} if s is None else dict(s)
    t1 = deref(t1, s); t2 = deref(t2, s)
    if isinstance(t1, Var):  return bind(t1.name, t2, s)
    if isinstance(t2, Var):  return bind(t2.name, t1, s)
    if isinstance(t1, Atom) and isinstance(t2, Atom):
        if t1.name != t2.name:
            raise UnifyError(f"atom mismatch: {t1.name} vs {t2.name}")
        return s
    if isinstance(t1, Fun) and isinstance(t2, Fun):
        if t1.name != t2.name or len(t1.args) != len(t2.args):
            raise UnifyError("functor mismatch or arity differs")
        for a,b in zip(t1.args, t2.args):
            s = unify(a, b, s)
        return s
    raise UnifyError("cannot unify terms")

In [ ]:
# ===============================
# Part 3 — Clauses (facts/rules) and knowledge base
# ===============================
@dataclass(frozen=True) class Fact: head: Fun
@dataclass(frozen=True) class Rule: head: Fun; body: Tuple[Fun, ...]
Clause = Fact | Rule

KB = List[Clause]

def vars_in_term(t: Term) -> Set[str]:
    if isinstance(t, Var): return {t.name}
    if isinstance(t, Atom): return set()
    if isinstance(t, Fun):
        out: Set[str] = set()
        for a in t.args: out |= vars_in_term(a)
        return out
    raise TypeError("unknown term")

def vars_in_clause(c: Clause) -> Set[str]:
    vs = set(vars_in_term(c.head))
    if isinstance(c, Rule):
        for g in c.body: vs |= vars_in_term(g)
    return vs

fresh_counter = 0
def freshen_var(name: str) -> str:
    global fresh_counter
    fresh_counter += 1
    return f"{name}_{fresh_counter}"

def standardize_apart(c: Clause) -> Clause:
    # Rename all variables in clause c to fresh ones to avoid capture
    mapping: Dict[str, Var] = {}
    def rename_term(t: Term) -> Term:
        t = t  # no deref here
        if isinstance(t, Var):
            if t.name not in mapping:
                mapping[t.name] = Var(freshen_var(t.name))
            return mapping[t.name]
        if isinstance(t, Atom):
            return t
        if isinstance(t, Fun):
            return Fun(t.name, tuple(rename_term(a) for a in t.args))
        raise TypeError("unknown term")
    if isinstance(c, Fact):
        return Fact(rename_term(c.head))
    else:
        return Rule(rename_term(c.head), tuple(rename_term(g) for g in c.body))

In [ ]:
# ===============================
# Part 4 — SLD resolution (depth-first, leftmost)
# ===============================
from typing import Generator

def solve(goals: List[Fun], kb: KB, s: Subst | None = None) -> Generator[Subst, None, None]:
    s = {} if s is None else dict(s)
    if not goals:
        yield s
        return
    (g, *rest) = goals
    # Try each clause: standardize apart, unify head with goal, then continue with body + rest
    for clause in kb:
        c = standardize_apart(clause)
        try:
            s1 = unify(g, c.head, s)
        except UnifyError:
            continue
        if isinstance(c, Fact):
            yield from solve(rest, kb, s1)
        else:
            new_goals = list(c.body) + rest
            yield from solve(new_goals, kb, s1)

def pretty_subst(s: Subst, vars_to_show: List[str]) -> str:
    items = []
    for v in vars_to_show:
        if v in s:
            items.append(f"{v} = {s[v]}")
    return "{ " + ", ".join(items) + " }"

# Term printers for nicer output
def tshow(t: Term) -> str:
    if isinstance(t, Var):  return t.name
    if isinstance(t, Atom): return t.name
    if isinstance(t, Fun):
        if t.name == "cons" and len(t.args)==2:
            # list sugar
            xs = []
            cur = t
            while isinstance(cur, Fun) and cur.name=="cons" and len(cur.args)==2:
                xs.append(tshow(cur.args[0]))
                cur = cur.args[1]
            if isinstance(cur, Atom) and cur.name == "nil":
                return "[" + ", ".join(xs) + "]"
        return f"{t.name}({', '.join(tshow(a) for a in t.args)})"
    return str(t)

In [ ]:
# ===============================
# Part 5 — Sample KB: family tree + ancestor/2
# ===============================
# Helpers to build terms quickly
def A(name: str) -> Atom: return Atom(name)
def V(name: str) -> Var:  return Var(name)
def F(name: str, *args: Term) -> Fun: return Fun(name, args)

# Facts: parent(X, Y)
kb: KB = [
    Fact(F("parent", A("alice"), A("bob"))),
    Fact(F("parent", A("bob"),   A("carol"))),
    Fact(F("parent", A("bob"),   A("dave"))),
    Fact(F("parent", A("carol"), A("erin"))),
]

# Rules:
# ancestor(X,Y) :- parent(X,Y).
# ancestor(X,Y) :- parent(X,Z), ancestor(Z,Y).
kb += [
    Rule(F("ancestor", V("X"), V("Y")), (F("parent", V("X"), V("Y")),)),
    Rule(F("ancestor", V("X"), V("Y")), (F("parent", V("X"), V("Z")), F("ancestor", V("Z"), V("Y")))),
]

# Query 1: Who are Alice's descendants?
q1 = [F("ancestor", A("alice"), V("Y"))]
answers = list(solve(q1, kb, {}))
print("Descendants of alice: ", sorted({tshow(ans['Y']) for ans in answers if 'Y' in ans}))

# Query 2: All ancestor pairs
q2 = [F("ancestor", V("X"), V("Y"))]
answers2 = list(solve(q2, kb, {}))
pairs = sorted({(tshow(ans['X']), tshow(ans['Y'])) for ans in answers2 if 'X' in ans and 'Y' in ans})
print("All ancestor pairs:", pairs)

In [ ]:
# ===============================
# Part 6 — Lists and append/3
# ===============================
nil = A("nil")
def cons(x: Term, xs: Term) -> Fun: return F("cons", x, xs)

# Facts and rules for append/3:
# append(nil, Ys, Ys).
# append(cons(X, Xs), Ys, cons(X, Zs)) :- append(Xs, Ys, Zs).
kb_append: KB = [
    Fact(F("append", nil, V("Ys"), V("Ys"))),
    Rule(F("append", cons(V("X"), V("Xs")), V("Ys"), cons(V("X"), V("Zs"))),
         (F("append", V("Xs"), V("Ys"), V("Zs")),))
]

# Example: append([1,2],[3,4], Zs)
list12 = cons(A("1"), cons(A("2"), nil))
list34 = cons(A("3"), cons(A("4"), nil))
q3 = [F("append", list12, list34, V("Zs"))]
ans3 = list(solve(q3, kb_append, {}))
print("append([1,2],[3,4],Zs) ->", [tshow(a['Zs']) for a in ans3])

### Your Turn (10–15 minutes)
1) Add a new relation `sibling(X,Y)` using `parent/2` (siblings share a parent, X != Y). Query for all sibling pairs.

2) Extend the KB with `grandparent(X,Y)` and verify answers.

3) Optional: Implement `member(Element, List)` using the `cons/nil` list encoding and query some examples.

### Reflection (2–3 sentences)
- In your words, how do **unification** and **SLD resolution** interact to answer a query?  
- Why is **standardize-apart** necessary when using a rule multiple times in the same proof tree?

In [ ]:
# Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")
submission = {
  "student_name": STUDENT_NAME,
  "student_id": STUDENT_ID,
  "timestamp": stamp,
  "checks": ["ancestor", "append"],
  "reflection": "(fill in here)"
}
with open("week8_submission.json", "w") as f:
  json.dump(submission, f, indent=2)
print("Saved week8_submission.json — upload with your notebook.")